In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import missingno as mgno


In [155]:
NSCH19 = pd.read_sas("data/nsch_2019e_topical.sas7bdat")
#NSCH22 = pd.read_sas("data/nsch_2022e_topical.sas7bdat")

In [64]:
geo_features = ['FIPSST']
## K7Q02R_R is number of days missed in school because of illness or injury.
## Responses are binned in intervals of 3 days, with 1 meaning no missed days, 5 meaning 11+ days, 6 meaning no enrolled
## in school
edu_features = ['K7Q02R_R']
med_features = ['K4Q27', #Was there any time when child needed health care but did not receive?
                #Note that if K4Q27 = 2 (No), then AVAILABLE to TRANSPORTCC are null

                'AVAILABLE', 'APPOINTMENT', 'ISSUECOST', 
                'NOTELIG', 'NOTOPEN', 'TRANSPORTCC',
                'CURRCOV', 
                'HOWMUCH', #how much was paid for health care
                'INSTYPE', #public/ priv/ none
                'K3Q04_R', #any type of health ins
                'K12Q12', #govt assistance plan (yes/ no)
                # Note that if CURRCOV = 2 (No), then K12Q12 is null

                'K3Q21B', #How often are cost reasonable?  If HOWMUCH=1, then K3Q21B is null
                'K3Q20', #How often does health insurance cover needs? If CURRCOV=2, then K3Q20 is null
                'K3Q22', #How often allowed to see providers?  If CURRCOV=2, then K3Q22 is null
                'K3Q25', #Problems Paying for Medical or Health Care
                'K4Q01', #is there place can take child when they are sick
                'K4Q24_R', #Did this child see a specialist?
                'K4Q26', #How difficult to get specialist care?  If K4Q24_R=3, then K4Q26 is null.
                
                'S4Q01', #Did this child visit a doctor?
                'K5Q31_R', #Provider Communication with School.  If S4Q01=2, then K5Q31_R is null.
                'K5Q32' #Satisfactory communication from doctor to school.  If S4Q01=2 or K5Q31_R=2 or 3, then K5Q32 is null.
                ]           

features = geo_features + edu_features + med_features

In [ ]:
def FIPS_to_State(data, state='both'):
    '''
    This function includes a column corresponding the state name and/ or abbreviation corresponding
    to the FIPS code

    Arguments:
    data = NSCH dataframe (in general, any dataframe with column 'FIPSST')
    state = 'abbr' for abbreviation only, 'full' for full state name, defaults to 'both'

    Returns: 
    dataframe with appended column(s) and FIPPST column changed to int type
    '''
    FIPS_state = pd.read_csv('data/FIPS_State.csv')
    FIPS_state['FIPSST'] = FIPS_state.FIPS

    data.FIPSST = data.FIPSST.apply(int)

    if state == 'abbr':
        data = data.merge(FIPS_state[['FIPSST', 'ABBR']], on='FIPSST')

    if state == 'full':
        data = data.merge(FIPS_state[['FIPSST', 'STATE']], on='FIPSST') 

    if state == 'both':
        data = data.merge(FIPS_state[['FIPSST', 'STATE', 'ABBR']], on='FIPSST')                

    return data


In [ ]:
## This calculates the null correlation between features
mgno.heatmap(NSCH19[features])

In [ ]:
## This will list the percent of data which is missing by state
## I borrowed the code from https://stackoverflow.com/questions/46106954/using-isnull-and-groupby-on-a-pandas-dataframe

NSCH19_nullByState = NSCH19[features].isnull().groupby(NSCH19['ABBR']).mean()
NSCH19_nullByState.sort_values(by="K7Q02R_R", ascending=False)


In [158]:
from sklearn.impute import SimpleImputer

def clean_NSCH(df, features, response = 'K7Q02R_R', 
               dropna_response = True, 
               cond_nans = True,
               rep_cond_nans = 0, 
               imputer = 'mode'):
    '''
    This function cleans up a dataframe in a similar style/ format of the NSCH data.  The reason for
    making a function is so that we can apply this generally to different years and to account for
    the train/ test split (e.g. imputation should be done on the training set to prevent data leakage).

    Arguments:
    df = NSCH dataframe
    features = list, a list of features we want to keep (including response variable)
    response = str, the response variable
    dropna_response = bool, drop nan from the response variable
    cond_nans = replaces conditional nans (i.e. those that depend on other feature values) 
                with rep_cond_nans value

                
    '''
    df = df[features]
    
    # Note: nans from the response variable should be dropped before data imputation
    if dropna_response == True: df = df[df[response].notna()]

    if cond_nans == True:
        for feat in ['AVAILABLE', 'APPOINTMENT', 'ISSUECOST', 
                'NOTELIG', 'NOTOPEN', 'TRANSPORTCC']:
            df.loc[NSCH19['K4Q27'] == 2, feat] = rep_cond_nans

        if 'K12Q12' in features: df.loc[NSCH19['CURRCOV'] == 2, 'K12Q12'] = rep_cond_nans
        if 'K3Q21B' in features: df.loc[NSCH19['HOWMUCH'] == 1, 'K3Q21B'] = rep_cond_nans        
        if 'K3Q20' in features: df.loc[NSCH19['CURRCOV'] == 2, 'K3Q20'] = rep_cond_nans        
        if 'K3Q22' in features: df.loc[NSCH19['CURRCOV'] == 2, 'K3Q22'] = rep_cond_nans  

        if 'K4Q26'and 'K4Q24_R' in features: df.loc[NSCH19['K4Q24_R'] == 3, 'K4Q26'] = rep_cond_nans
        if 'K5Q31_R'and 'S4Q01' in features: df.loc[NSCH19['S4Q01'] == 2, 'K5Q31_R'] = rep_cond_nans

        if 'K5Q32'and 'S4Q01' in features: df.loc[NSCH19['S4Q01'] == 2, 'K5Q32'] = rep_cond_nans
        if 'K5Q32'and 'K5Q31_R' in features: df.loc[NSCH19['K5Q31_R'] == 2, 'K5Q32'] = rep_cond_nans
        if 'K5Q32'and 'K5Q31_R' in features: df.loc[NSCH19['K5Q31_R'] == 3, 'K5Q32'] = rep_cond_nans 

    # This imputes nan entries by mode
    if imputer == 'mode':
        imp_mode = SimpleImputer(missing_values = np.nan, strategy='most_frequent')
        for col in df.columns:
            if df[col].isnull().sum() != 0:
                imp_col = imp_mode.fit_transform(df[col].values.reshape(-1,1))
                df[col] = imp_col


    # Adding a state name and state abbreviation column corresponding to the FIPS code
    df = FIPS_to_State(df)
    
    return df

In [125]:
clean_NSCH19 = clean_NSCH(NSCH19, features)


In [159]:
imp_NSCH19 = clean_NSCH(NSCH19, features)

In [126]:
clean_NSCH19.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21021 entries, 0 to 21020
Data columns (total 26 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   FIPSST       21021 non-null  int64  
 1   K7Q02R_R     21021 non-null  float64
 2   K4Q27        20958 non-null  float64
 3   AVAILABLE    20953 non-null  float64
 4   APPOINTMENT  20953 non-null  float64
 5   ISSUECOST    20953 non-null  float64
 6   NOTELIG      20953 non-null  float64
 7   NOTOPEN      20953 non-null  float64
 8   TRANSPORTCC  20953 non-null  float64
 9   CURRCOV      20973 non-null  float64
 10  HOWMUCH      20834 non-null  float64
 11  INSTYPE      20739 non-null  float64
 12  K3Q04_R      20970 non-null  float64
 13  K12Q12       20917 non-null  float64
 14  K3Q21B       20792 non-null  float64
 15  K3Q20        20893 non-null  float64
 16  K3Q22        20895 non-null  float64
 17  K3Q25        16171 non-null  float64
 18  K4Q01        20944 non-null  float64
 19  K4Q2

In [160]:
imp_NSCH19.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21021 entries, 0 to 21020
Data columns (total 26 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   FIPSST       21021 non-null  int64  
 1   K7Q02R_R     21021 non-null  float64
 2   K4Q27        21021 non-null  float64
 3   AVAILABLE    21021 non-null  float64
 4   APPOINTMENT  21021 non-null  float64
 5   ISSUECOST    21021 non-null  float64
 6   NOTELIG      21021 non-null  float64
 7   NOTOPEN      21021 non-null  float64
 8   TRANSPORTCC  21021 non-null  float64
 9   CURRCOV      21021 non-null  float64
 10  HOWMUCH      21021 non-null  float64
 11  INSTYPE      21021 non-null  float64
 12  K3Q04_R      21021 non-null  float64
 13  K12Q12       21021 non-null  float64
 14  K3Q21B       21021 non-null  float64
 15  K3Q20        21021 non-null  float64
 16  K3Q22        21021 non-null  float64
 17  K3Q25        21021 non-null  float64
 18  K4Q01        21021 non-null  float64
 19  K4Q2

In [90]:
NSCH19_nullByState = clean_NSCH19.isnull().groupby(clean_NSCH19['STATE']).mean()

NSCH19_nullByState.sort_values(by="K3Q25", ascending=False)

,FIPSST,K7Q02R_R,K4Q27,AVAILABLE,APPOINTMENT,ISSUECOST,NOTELIG,NOTOPEN,TRANSPORTCC,CURRCOV,...,K3Q22,K3Q25,K4Q01,K4Q24_R,K4Q26,S4Q01,K5Q31_R,K5Q32,STATE,ABBR
STATE,,,,,,,,,,,,,,,,,,,,,
New Mexico,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.006466,0.364224,0.004310,0.012931,0.010776,0.000000,0.019397,0.021552,0.0,0.0
Mississippi,0.0,0.0,0.015660,0.013423,0.013423,0.013423,0.013423,0.013423,0.013423,0.011186,...,0.008949,0.362416,0.006711,0.008949,0.008949,0.002237,0.031320,0.035794,0.0,0.0
Oklahoma,0.0,0.0,0.002198,0.004396,0.004396,0.004396,0.004396,0.004396,0.004396,0.002198,...,0.002198,0.336264,0.002198,0.006593,0.010989,0.002198,0.013187,0.013187,0.0,0.0
Louisiana,0.0,0.0,0.006865,0.009153,0.009153,0.009153,0.009153,0.009153,0.009153,0.002288,...,0.006865,0.331808,0.002288,0.004577,0.013730,0.002288,0.011442,0.011442,0.0,0.0
Alaska,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002358,...,0.004717,0.311321,0.000000,0.002358,0.004717,0.000000,0.009434,0.009434,0.0,0.0
Arkansas,0.0,0.0,0.004338,0.004338,0.004338,0.004338,0.004338,0.004338,0.004338,0.002169,...,0.000000,0.303688,0.000000,0.004338,0.008677,0.000000,0.019523,0.019523,0.0,0.0
New York,0.0,0.0,0.004739,0.004739,0.004739,0.004739,0.004739,0.004739,0.004739,0.004739,...,0.018957,0.298578,0.004739,0.009479,0.014218,0.000000,0.007109,0.007109,0.0,0.0
Kentucky,0.0,0.0,0.005141,0.005141,0.005141,0.005141,0.005141,0.005141,0.005141,0.000000,...,0.002571,0.290488,0.000000,0.007712,0.012853,0.000000,0.007712,0.007712,0.0,0.0
Hawaii,0.0,0.0,0.005831,0.005831,0.005831,0.005831,0.005831,0.005831,0.005831,0.002915,...,0.017493,0.288630,0.002915,0.005831,0.008746,0.000000,0.008746,0.008746,0.0,0.0


In [154]:
imp_mode = SimpleImputer(strategy='most_frequent')

imp_col = imp_mode.fit_transform(NSCH19.AVAILABLE.values.reshape(-1,1))

NSCH19['AVAILABLE'] = imp_col
NSCH19.AVAILABLE

0        2.0
1        2.0
2        2.0
3        2.0
4        2.0
        ... 
29428    2.0
29429    2.0
29430    2.0
29431    2.0
29432    2.0
Name: AVAILABLE, Length: 29433, dtype: float64